In [2]:
#
# This is a sample Notebook to demonstrate how to read "MNIST Dataset"
#
import numpy as np # linear algebra
import struct
from array import array
from os.path  import join

#
# MNIST Data Loader Class
#
class MnistDataloader(object):
    def __init__(self, training_images_filepath,training_labels_filepath,
                 test_images_filepath, test_labels_filepath):
        self.training_images_filepath = training_images_filepath
        self.training_labels_filepath = training_labels_filepath
        self.test_images_filepath = test_images_filepath
        self.test_labels_filepath = test_labels_filepath
    
    def read_images_labels(self, images_filepath, labels_filepath):        
        labels = []
        with open(labels_filepath, 'rb') as file:
            magic, size = struct.unpack(">II", file.read(8))
            if magic != 2049:
                raise ValueError('Magic number mismatch, expected 2049, got {}'.format(magic))
            labels = array("B", file.read())        
        
        with open(images_filepath, 'rb') as file:
            magic, size, rows, cols = struct.unpack(">IIII", file.read(16))
            if magic != 2051:
                raise ValueError('Magic number mismatch, expected 2051, got {}'.format(magic))
            image_data = array("B", file.read())        
        images = []
        for i in range(size):
            images.append([0] * rows * cols)
        for i in range(size):
            img = np.array(image_data[i * rows * cols:(i + 1) * rows * cols])
            img = img.reshape(28, 28)
            images[i][:] = img            
        
        return images, labels
            
    def load_data(self):
        x_train, y_train = self.read_images_labels(self.training_images_filepath, self.training_labels_filepath)
        x_test, y_test = self.read_images_labels(self.test_images_filepath, self.test_labels_filepath)
        return (x_train, y_train),(x_test, y_test)        
#
# Verify Reading Dataset via MnistDataloader class
#

import random
import matplotlib.pyplot as plt

#
# Set file paths based on added MNIST Datasets
#
# input_path = '../input'
# training_images_filepath = join(input_path, 'train-images-idx3-ubyte/train-images-idx3-ubyte')
# training_labels_filepath = join(input_path, 'train-labels-idx1-ubyte/train-labels-idx1-ubyte')
# test_images_filepath = join(input_path, 't10k-images-idx3-ubyte/t10k-images-idx3-ubyte')
# test_labels_filepath = join(input_path, 't10k-labels-idx1-ubyte/t10k-labels-idx1-ubyte')

# training_images_filepath = r"C:\Users\ujjwal\Desktop\sml\train-images.idx3-ubyte"
# training_labels_filepath = r"C:\Users\ujjwal\Desktop\sml\train-labels.idx1-ubyte"
# test_images_filepath = r"C:\Users\ujjwal\Desktop\sml\t10k-images.idx3-ubyte"
# test_labels_filepath = r"C:\Users\ujjwal\Desktop\sml\t10k-labels.idx1-ubyte"

training_images_filepath = r"train-images.idx3-ubyte"
training_labels_filepath = r"train-labels.idx1-ubyte"
test_images_filepath = r"t10k-images.idx3-ubyte"
test_labels_filepath = r"t10k-labels.idx1-ubyte"



#
# Helper function to show a list of images with their relating titles
#
def show_images(images, title_texts):
    cols = 5
    rows = int(len(images)/cols) + 1
    plt.figure(figsize=(30,20))
    index = 1    
    for x in zip(images, title_texts):        
        image = x[0]        
        title_text = x[1]
        plt.subplot(rows, cols, index)        
        plt.imshow(image, cmap=plt.cm.gray)
        if (title_text != ''):
            plt.title(title_text, fontsize = 15);        
        index += 1

#
# Load MINST dataset
#
mnist_dataloader = MnistDataloader(training_images_filepath, training_labels_filepath, test_images_filepath, test_labels_filepath)
(training_images, training_labels), (test_images, test_labels) = mnist_dataloader.load_data()

print(type(training_images))



training_images = np.array(training_images)
test_images = np.array(test_images)
training_labels = np.array(training_labels)
test_labels = np.array(test_labels)


print((training_images.shape))
print(type(training_images))
#this data is not flattened therefore 




import numpy as np

# training_images[0] = np.array(training_images[0])  # Convert list of arrays to a single NumPy array

# print(training_images[0])  # Should print (28, 28)

# plt.imshow(np.array(x_train[-1]), cmap="gray")
# plt.show()

print(f"Training images shape: {training_images.shape}")
print(f"Training labels shape: {training_labels.shape}")
print(f"Test images shape: {test_images.shape}")
print(f"Test labels shape: {test_labels.shape}")


train_index_0 = np.where(training_labels==0)[0]
train_index_1 = np.where(training_labels==1)[0]

train_index_0=train_index_0[:1000]
train_index_1=train_index_1[:1000]

train_indices = np.concatenate([train_index_0, train_index_1])

X_train = training_images[train_indices]
y_train = training_labels[train_indices]

test_indices = np.where((test_labels == 0) | (test_labels == 1))[0]
X_test = test_images[test_indices]
y_test = test_labels[test_indices]

X_train_flat = X_train.reshape(X_train.shape[0], -1)
X_test_flat = X_test.reshape(X_test.shape[0], -1)





                                           



<class 'list'>
(60000, 28, 28)
<class 'numpy.ndarray'>
Training images shape: (60000, 28, 28)
Training labels shape: (60000,)
Test images shape: (10000, 28, 28)
Test labels shape: (10000,)


In [3]:
def manual_pca(X, n_components=5):
    # Center the data
    X_centered = X - np.mean(X, axis=0)
    
    cov_matrix = np.cov(X_centered, rowvar=False)
    
    eigenvalues, eigenvectors = np.linalg.eigh(cov_matrix)
    
    idx = np.argsort(eigenvalues)[::-1]
    eigenvalues = eigenvalues[idx]
    eigenvectors = eigenvectors[:, idx]
    
    components = eigenvectors[:, :n_components]
    
    X_pca = np.dot(X_centered, components)
    
    return X_pca, components, eigenvalues[:n_components]


X_train_pca, pca_components, explained_var = manual_pca(X_train_flat, n_components=5)
X_test_pca = np.dot(X_test_flat - np.mean(X_train_flat, axis=0), pca_components)

print(X_train_pca.shape)
print(y_train.shape)

(2000, 5)
(2000,)


In [4]:
class DecisionStump:
    def __init__(self):
        self.feature_index = None
        self.threshold = None
        self.direction = 1  # 1 for < threshold = -1, else +1; -1 for reverse

    def predict(self, X):
        n_samples = X.shape[0]
        predictions = np.ones(n_samples)
        if self.direction == 1:
            predictions[X[:, self.feature_index] < self.threshold] = -1
        else:
            predictions[X[:, self.feature_index] >= self.threshold] = -1
        return predictions


In [5]:

class AdaBoost:
    def __init__(self, total_rounds=200):
        self.total_rounds = total_rounds
        self.stumps = []
        self.stump_weights = []
        self.train_errors = []

    def fit(self, X, y):
        n_samples, n_features = X.shape
        y = np.where(y == 0, -1, 1)  # Convert labels to -1 and +1
        sample_weights = np.ones(n_samples) / n_samples

        for _ in range(self.total_rounds):
            best_stump = None
            best_error = float('inf')

            for feature_index in range(n_features):
                feature_values = X[:, feature_index]
                thresholds = np.linspace(feature_values.min(), feature_values.max(), 3)

                for threshold in thresholds:
                    for direction in [1, -1]:
                        stump = DecisionStump()
                        stump.feature_index = feature_index
                        stump.threshold = threshold
                        stump.direction = direction

                        preds = stump.predict(X)
                        misclassified = (preds != y)
                        error = np.sum(sample_weights * misclassified)

                        if error < best_error:
                            best_error = error
                            best_stump = stump

            # Compute alpha (stump weight)
            eps = 1e-10  # To avoid division by zero
            alpha = 0.5 * np.log((1 - best_error) / (best_error + eps))

            # Update sample weights
            preds = best_stump.predict(X)
            sample_weights *= np.exp(-alpha * y * preds)
            sample_weights /= np.sum(sample_weights)

            # Store stump and its weight
            self.stumps.append(best_stump)
            self.stump_weights.append(alpha)
            self.train_errors.append(best_error)

    def predict(self, X):
        stump_preds = np.array([alpha * stump.predict(X) for stump, alpha in zip(self.stumps, self.stump_weights)])
        y_pred = np.sign(np.sum(stump_preds, axis=0))
        y_pred[y_pred == 0] = 1  # Default to +1 if sign == 0
        return y_pred


In [6]:
# Convert labels to binary (0 -> -1, 1 -> 1)
y_train_boost = np.where(y_train == 0, -1, 1)
y_test_boost = np.where(y_test == 0, -1, 1)

# Initialize and train AdaBoost
adaboost = AdaBoost()
adaboost.fit(X_train_pca, y_train_boost)

# Predictions

train_preds = adaboost.predict(X_train_pca)
test_preds = adaboost.predict(X_test_pca)

# Accuracy
train_accuracy = np.mean(train_preds == y_train_boost)
test_accuracy = np.mean(test_preds == y_test_boost)

print(f"Training Accuracy: {train_accuracy:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

Training Accuracy: 0.5000
Test Accuracy: 0.5371
